In [ ]:
using Knet, Test, Base.Iterators, Printf, LinearAlgebra, Random, CUDA, IterTools, DelimitedFiles, Statistics
Knet.atype() = KnetArray{Float32}
using Images, TestImages, OffsetArrays, Colors
using Plots

In [ ]:
include("models/transformer.jl")
include("dataloader/dataloader.jl")

In [ ]:
# read  files
annotations = readdlm("archive/annotations_train.txt", '\t', String, '\n')[2:end,2:end]

# prepare vocab object
bow = ""
for sent in annotations[:,2]#
    bow *= sent
end
#bow = split(bow)
# bow wil be one giant sentence, corpus.
v = Vocab(bow)

# # Initialize Iterators
# img_iterator = ImgReader("archive/img_dirs_val.txt",load_and_process)
# tgt_iterator = TextReader("archive/ann_caps_val.txt",v)

# # simulate retrieving one batch
# img_state, tgt_state = nothing, nothing
# flag = true
# iter = 1
# batch = get_next_batch(img_iterator, tgt_iterator, img_state, tgt_state)

# batch_imgs, batch_indices, img_state, tgt_state = batch
# batch_imgs, batch_indices, labels = prepare_batch(batch, tgt_iterator)
# println(iter)

In [ ]:
pretrained = Knet.load("ckpts2/transformer_mlp_512_epoch100.jld2","transformer")

In [ ]:
evalon = "val"

In [ ]:
# ONE VALIDATION EPOCH LOOP
pred_corpus = []
img_state, tgt_state = nothing, nothing
img_iterator = ImgReader("archive/img_dirs_$evalon.txt",load_and_process)
tgt_iterator = TextReader("archive/ann_caps_$evalon.txt",v)
while true #iter < 100 #flag !== nothing
    batch = get_next_batch(img_iterator, tgt_iterator, img_state, tgt_state)
    if batch == nothing
        break
    end
    batch_imgs, batch_indices, img_state, tgt_state = batch
    batch_imgs, batch_indices, labels = prepare_batch(batch, tgt_iterator)
    batch_imgs_nomalized = 2.0 .* batch_imgs .- 1.0

    word_probs = pretrained(batch_imgs_nomalized, batch_indices)
    
    argmaxprobs = Array(mapslices(argmax,Array(word_probs),dims=1)[1,:,:])
    pred_words = tgt_iterator.vocab.i2w[argmaxprobs]
    pred_words = permutedims(pred_words,(2,1)) # each sentence prediction is now a row
    
    push!(pred_corpus, pred_words)   

end

In [ ]:
open("bleu_files/pred_corpus.txt", "w") do file
    for batch_corpus in pred_corpus
        for sent_i in 1:size(batch_corpus,1)
            for word in batch_corpus[sent_i,:]
                print(file, word, ' ')
            end
            print(file, '\n')
        end
    end
end

In [ ]:
# read  files
annotations = readdlm("archive/annotations_$evalon.txt", '\t', String, '\n')[2:end,2:end]

# prepare vocab object
ref_corpus=[]
for sent in annotations[:,2]#
    push!(ref_corpus, sent)
end

open("bleu_files/ref_corpus.txt", "w") do file2
    
    for sent in ref_corpus
        println(file2, sent)
    end
end